In [28]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col, broadcast

# Load environment variables from .env file
load_dotenv()

# Get AWS credentials from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Check if AWS credentials are present
if not aws_access_key_id or not aws_secret_access_key:
    raise ValueError("AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY are required.")

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Filter Invalid Transctions") \
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.master", "spark://spark-master-2:7077") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.sql.shuffle.partitions", "62") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Define schema for the 'trans' and 'account' tables
trans_schema = StructType([
    StructField("trans_id", IntegerType(), True),
    StructField("account_id", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("type", StringType(), True),
    StructField("operation", StringType(), True),
    StructField("amount", FloatType(), True),
    StructField("balance", FloatType(), True),
    StructField("k_symbol", StringType(), True),
    StructField("branch", StringType(), True),
    StructField("bank", StringType(), True),
    StructField("account", IntegerType(), True)
])

account_schema = StructType([
    StructField("account_id", IntegerType(), True),
    StructField("district_id", IntegerType(), True),
    StructField("frequency", StringType(), True),
    StructField("date", StringType(), True)
])

# Read the 'trans' and 'account' CSV files select for just necessary coloumns 

trans_df = spark.read \
    .option("delimiter", ";") \
    .option("header", "true") \
    .schema(trans_schema) \
    .csv("s3a://nmourmx-scigility/Bronze/trans/trans.csv") \
    .select("trans_id", "account_id", "amount", "balance", "date", "type", "operation")

account_df = spark.read \
    .option("delimiter", ";") \
    .option("header", "true") \
    .schema(account_schema) \
    .csv("s3a://nmourmx-scigility/Bronze/account/account.csv") \
    .select("account_id", "district_id")


# Ensure both use the same data type
### repartition based on the account_id to optimize future joins 
trans_df = trans_df.withColumn("account_id", col("account_id").cast("int")) \
                   .repartition(100, "account_id") 
account_df = account_df.withColumn("account_id", col("account_id").cast("int"))


# Cache the DataFrames to improve performance
trans_df = trans_df.cache()
account_df = account_df.cache()




# Show the first few rows of both DataFrames for validation
trans_df.show(2)
account_df.show(2)


# Count the number of rows in each DataFrame
trans_count = trans_df.count()
print(f"Number of rows in 'trans_df': {trans_count}")


account_count = account_df.count()
print(f"Number of rows in 'account_df': {account_count}")

# Show the filtered transactions (valid account_id only)

# Perform the join and broadcast the smaller account dataframe
# valid_trans_df = trans_df.join(broadcast(account_df), "account_id", "inner")
# Perform optimized broadcast join
valid_trans_df = trans_df.join(broadcast(account_df), "account_id", "inner") \
                         .persist(StorageLevel.MEMORY_AND_DISK)

# Show sample only (avoid full count unless required)
print("Sample valid transactions:")
valid_trans_df.show(5)





valid_trans_count_df = valid_trans_df.count()
print(f"Number of rows in 'valid_trans_df': {valid_trans_count_df}")
valid_trans_df.show(2)

# Persist the DataFrame with valid transactions
valid_trans_df = valid_trans_df.persist(StorageLevel.MEMORY_AND_DISK)
# Coalesce the DataFrame before saving to reduce the number of partitions
valid_trans_df_coalesced = valid_trans_df.coalesce(1)


# Show the filtered invalid transactions
invalid_trans_df = trans_df.join(account_df, on="account_id", how="left_anti")
print(f"Number of transactions with invalid account_id: {invalid_trans_df.count()}")
invalid_trans_df.show(2)

# OPTIONAL: Write to S3 (without coalesce for parallel write)
# valid_trans_df.write \
#     .mode("overwrite") \
#     .parquet("s3a://nmourmx-scigility/Silver/valid_trans_parquet/")


# Save the filtered DataFrame to S3 in the Silver folder as Parquet
# valid_trans_df_coalesced.write \
#     .mode("overwrite") \
#     .parquet("s3a://nmourmx-scigility/Silver/valid_trans_parquet/")

# Stop the Spark session
spark.stop()


+--------+----------+------+-------+----------+------+---------+
|trans_id|account_id|amount|balance|      date|  type|operation|
+--------+----------+------+-------+----------+------+---------+
|  971490|      1645|  43.0|19460.0|1996-11-30|PRIJEM|     NULL|
|  589995|      3273|  15.0|26679.0|1997-04-30| VYDAJ|    VYBER|
+--------+----------+------+-------+----------+------+---------+
only showing top 2 rows



+----------+-----------+
|account_id|district_id|
+----------+-----------+
|         1|         18|
|         2|          1|
+----------+-----------+
only showing top 2 rows



Number of rows in 'trans_df': 1056410
Number of rows in 'account_df': 4500
Sample valid transactions:


+----------+--------+------+-------+----------+------+--------------+-----------+
|account_id|trans_id|amount|balance|      date|  type|     operation|district_id|
+----------+--------+------+-------+----------+------+--------------+-----------+
|      1645|  971490|  43.0|19460.0|1996-11-30|PRIJEM|          NULL|         50|
|      3273|  589995|  15.0|26679.0|1997-04-30| VYDAJ|         VYBER|         74|
|      1580|  285772|8400.0|41404.0|1998-04-12| VYDAJ|         VYBER|         69|
|      2366|  425228| 388.0|24110.0|1998-02-07| VYDAJ|PREVOD NA UCET|         68|
|      4519|  745408|2100.0|26200.0|1994-12-05|PRIJEM|         VKLAD|          8|
+----------+--------+------+-------+----------+------+--------------+-----------+
only showing top 5 rows



Number of rows in 'valid_trans_df': 1056320
+----------+--------+------+-------+----------+------+---------+-----------+
|account_id|trans_id|amount|balance|      date|  type|operation|district_id|
+----------+--------+------+-------+----------+------+---------+-----------+
|      1645|  971490|  43.0|19460.0|1996-11-30|PRIJEM|     NULL|         50|
|      3273|  589995|  15.0|26679.0|1997-04-30| VYDAJ|    VYBER|         74|
+----------+--------+------+-------+----------+------+---------+-----------+
only showing top 2 rows



25/07/31 21:35:07 WARN CacheManager: Asked to cache already cached data.


Number of transactions with invalid account_id: 90
+----------+--------+------+-------+----------+-----+---------+
|account_id|trans_id|amount|balance|      date| type|operation|
+----------+--------+------+-------+----------+-----+---------+
|     11500|  580601|4186.0|30123.0|1998-05-09|VYDAJ|    VYBER|
|     11639|  858463|7753.0|22856.0|1994-11-07|VYDAJ|    VYBER|
+----------+--------+------+-------+----------+-----+---------+
only showing top 2 rows



In [26]:
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col, broadcast

# Load environment variables
load_dotenv()

aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

if not aws_access_key_id or not aws_secret_access_key:
    raise ValueError("AWS credentials are missing")

# Initialize Spark session with performance tuning
spark = SparkSession.builder \
    .appName("Filter Invalid Transactions") \
    .config("spark.master", "spark://spark-master-2:7077") \
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.sql.shuffle.partitions", "62") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Define schemas
trans_schema = StructType([
    StructField("trans_id", IntegerType(), True),
    StructField("account_id", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("type", StringType(), True),
    StructField("operation", StringType(), True),
    StructField("amount", FloatType(), True),
    StructField("balance", FloatType(), True),
    StructField("k_symbol", StringType(), True),
    StructField("branch", StringType(), True),
    StructField("bank", StringType(), True),
    StructField("account", IntegerType(), True)
])

account_schema = StructType([
    StructField("account_id", IntegerType(), True),
    StructField("district_id", IntegerType(), True),
    StructField("frequency", StringType(), True),
    StructField("date", StringType(), True)
])

# Read only necessary columns from CSV
trans_df = spark.read \
    .option("delimiter", ";") \
    .option("header", "true") \
    .schema(trans_schema) \
    .csv("s3a://nmourmx-scigility/Bronze/trans/trans.csv") \
    .select("trans_id", "account_id", "amount", "balance", "date", "type", "operation")

account_df = spark.read \
    .option("delimiter", ";") \
    .option("header", "true") \
    .schema(account_schema) \
    .csv("s3a://nmourmx-scigility/Bronze/account/account.csv") \
    .select("account_id", "district_id")

# Ensure both use the same data type
trans_df = trans_df.withColumn("account_id", col("account_id").cast("int")) \
                   .repartition(100, "account_id")  # Efficient shuffling
account_df = account_df.withColumn("account_id", col("account_id").cast("int"))

# Perform optimized broadcast join
valid_trans_df = trans_df.join(broadcast(account_df), "account_id", "inner") \
                         .persist(StorageLevel.MEMORY_AND_DISK)

# Show sample only (avoid full count unless required)
print("Sample valid transactions:")
valid_trans_df.show(5)

# Detect invalid transactions (those with account_id not in account_df)
invalid_trans_df = trans_df.join(account_df, "account_id", "left_anti")

print(f"Invalid transactions count: {invalid_trans_df.count()}")
invalid_trans_df.show(5)

# OPTIONAL: Write to S3 (without coalesce for parallel write)
# valid_trans_df.write \
#     .mode("overwrite") \
#     .parquet("s3a://nmourmx-scigility/Silver/valid_trans_parquet/")

spark.stop()


Sample valid transactions:


+----------+--------+------+-------+----------+------+--------------+-----------+
|account_id|trans_id|amount|balance|      date|  type|     operation|district_id|
+----------+--------+------+-------+----------+------+--------------+-----------+
|      1645|  971490|  43.0|19460.0|1996-11-30|PRIJEM|          NULL|         50|
|      3273|  589995|  15.0|26679.0|1997-04-30| VYDAJ|         VYBER|         74|
|      1580|  285772|8400.0|41404.0|1998-04-12| VYDAJ|         VYBER|         69|
|      2366|  425228| 388.0|24110.0|1998-02-07| VYDAJ|PREVOD NA UCET|         68|
|      4519|  745408|2100.0|26200.0|1994-12-05|PRIJEM|         VKLAD|          8|
+----------+--------+------+-------+----------+------+--------------+-----------+
only showing top 5 rows



Invalid transactions count: 90


+----------+--------+------+-------+----------+------+---------+
|account_id|trans_id|amount|balance|      date|  type|operation|
+----------+--------+------+-------+----------+------+---------+
|     11500|  580601|4186.0|30123.0|1998-05-09| VYDAJ|    VYBER|
|     11639|  858463|7753.0|22856.0|1994-11-07| VYDAJ|    VYBER|
|     12139|  694873|4550.0| 9642.0|1998-08-22| VYDAJ|    VYBER|
|     12100|  869665|9625.0|39800.0|1997-08-31| VYDAJ|    VYBER|
|     11928|  939364|2148.0|73450.0|1993-02-28|PRIJEM|     NULL|
+----------+--------+------+-------+----------+------+---------+
only showing top 5 rows

